# 19.模型预测控制（model predictive control，MPC）算法
1. 基于值函数的方法 DQN、基于策略的方法 REINFORCE 以及两者结合的方法 Actor-Critic，都是 **无模型（model-free）** 的方法，没有建立一个 **环境模型** 来帮助智能体决策。
2. 在 **深度强化学习** 领域下，**基于模型（model-based）** 的方法不仅依赖于与 **真实环境** 交互获取的奖励信号，还试图用 **神经网络** 学习一个 **环境模型**，通过这个模型来**预测**环境的动态和奖励来帮助**智能体**训练和决策。
3. 利用 **环境模型** 帮助智能体训练和决策的方法有很多种，例如类似之前 [Dyna](../RL_Fundamentals/7.Dyna-Q算法.ipynb) 的思想：生成一些数据来加入策略训练中。
> **模型预测控制（MPC）** 算法：思想源于控制理论中的 **最优控制（Optimal Control）** + **动态规划（Dynamic Programming）**，结合了系统动态建模、实时优化和反馈控制。
> **MPC** 并不构建一个显式的策略，而是使用 **环境模型** 预测未来一段时间内环境系统的状态变化，从而选择 **当前步** 要采取的动作。

> **MPC** 方法在 **每次采取动作时**，首先会生成一些 **候选动作序列**（生成候选动作序列的过程被称为 **打靶(shooting)**），然后根据历史数据学习得到的 **环境模型$\hat{P}(s,a)$** 来确定每一条候选序列能得到多好的结果，最终选择结果最好的那条动作序列的 **第一个动作** 来执行：
$$\arg\max_{a_{k:k+H}}\sum_{t=k}^{k+H}r(s_t,a_t)\mathrm{~s.t.~}s_{t+1}=\hat{P}(s_t,a_t)$$
- $H$为推演的长度
- $\arg\max_{a_{k:k+H}}$表示从所有动作序列中选取累积奖励最大的序列

## 19.1 候选动作序列的生成：交叉熵方法（cross entropy method，CEM）
> **MPC** 方法中的一个关键是如何生成一些 **候选动作序列**。
> 常见的 **随机打靶法（random shooting method）** 随机生成$N$条长度为$H$的动作序列，对于一些简单的环境，这个方法不但十分简单，而且效果还不错。

*相比于 **随机打靶法**，**交叉熵方法** 通过维护一个 **带参数的分布**，能够利用之前采样到的比较好的结果 **更新分布中的参数**，**提高**分布中能获得较高累积奖励的动作序列的**概率**，在一定程度上减少采样到一些较差动作的概率，从而使得算法更加高效*：
> 为了实现 **交叉熵方法**，**带参数的分布** 将采用 **截断正态分布（Truncated Normal Distribution）** ：在标准正态分布的基础上，对其定义域进行 **截断**，使得超出指定区间的部分被剪切掉，区间外的概率密度为零，但概率密度函数在指定的截断区间内被规范化积分依然为 1。由于截断了分布的尾部，**截断正态分布** 的分布范围会变得更加集中，从而做到了**提高**高奖励动作序列**概率**的效果。

In [3]:
import numpy as np
from scipy.stats import truncnorm  # 用于生成和操作截断正态分布（Truncated Normal Distribution）

class CEM:  # 优化随机采样动作的分布的均值和方差
    def __init__(self, n_sequence, elite_ratio, fake_env, upper_bound, lower_bound):
        self.n_sequence = n_sequence  # 生成的动作序列数量
        self.elite_ratio = elite_ratio  # 精英策略的比例（选择最好的前elite_ratio部分）
        self.upper_bound = upper_bound
        self.lower_bound = lower_bound
        self.fake_env = fake_env  # 用来模拟环境的虚拟环境对象

    def optimize(self, state, init_mean, init_var):
        mean, var = init_mean, init_var  # 初始的动作分布的均值和方差，mean 控制动作的中心位置，var 控制动作的分散程度
        X = truncnorm(-2, 2, loc=np.zeros_like(mean), scale=np.ones_like(var))  # 创建n维（动作维度）截断正态分布对象
        state = np.tile(state, (self.n_sequence, 1))  # 扩展 n_sequence 个相同的状态，以便在同一时刻生成多个动作序列

        for _ in range(5):  # 重复进行多次优化
            lb_dist, ub_dist = mean - self.lower_bound, self.upper_bound - mean
            constrained_var = np.minimum(np.minimum(np.square(lb_dist / 2), np.square(ub_dist / 2)), var)  # 方差的约束
            # 生成动作序列
            action_sequences = [X.rvs() for _ in range(self.n_sequence)] * np.sqrt(constrained_var) + mean
            # 计算每条动作序列的累积奖励
            returns = self.fake_env.propagate(state, action_sequences)[:, 0]
            # 选取累积奖励高的若干条动作序列（精英序列）
            elites = action_sequences[np.argsort(returns)][-int(self.elite_ratio * self.n_sequence):]
            new_mean = np.mean(elites, axis=0)
            new_var = np.var(elites, axis=0)
            # 指数加权平均平滑更新更新动作序列分布
            mean = 0.1 * mean + 0.9 * new_mean
            var = 0.1 * var + 0.9 * new_var

        return mean

In [4]:
import numpy as np
from scipy.stats import truncnorm

# 模拟 CEM 中的参数
n_sequence = 5
action_dim = 3
mean = np.array([0.5, -0.2, 1.0])
var = np.array([0.1, 0.2, 0.3])

# 单条动作序列长度控制
mean = np.tile(mean,2)
var = np.tile(var,2)

# 截断正态分布对象
X = truncnorm(-2, 2, loc=np.zeros_like(mean), scale=np.ones_like(var))

# 原代码生成动作序列
action_sequences = [X.rvs() for _ in range(n_sequence)]

print("类型:", type(action_sequences))
print("内容:", action_sequences)
print("形状:", np.array(action_sequences).shape)


类型: <class 'list'>
内容: [array([ 0.81457096, -1.1385638 , -1.25696675, -1.13795376, -0.2855569 ,
        0.68330365]), array([-0.71508926, -1.97032919, -0.47137062, -0.72447478, -0.24919709,
        0.06653321]), array([ 0.29918645,  0.41406621,  0.04763129,  0.70885207, -0.44162991,
        0.1202528 ]), array([-0.58776573, -1.12610308, -0.50789616,  0.31755594, -0.06837855,
       -0.41974492]), array([ 0.69528134, -0.16629011,  0.02383139, -1.61037627, -0.85690703,
       -0.10051585])]
形状: (5, 6)


## 19.2 模型的构建
> **候选动作序列** 生成好之后，便是利用构建的**环境模型**，对每条序列进行评估。
> 在强化学习中，与智能体交互的 **真实环境** 是一个动态系统，所以拟合它的 **环境模型** 也通常是一个 **动态模型**。一般认为一个系统中存在**两种不确定性**： **偶然不确定性（aleatoric uncertainty）** 和 **认知不确定性（epistemic uncertainty）**：
![偶然不确定性和认知不确定性](Illustrations/偶然不确定性和认知不确定性.png)
- **偶然不确定性** 是由于系统中本身存在的随机性引起的
- **认知不确定性** 是由“见”过的数据较少导致的自身认知的不足而引起的

> **带有轨迹采样的概率集成（probabilistic ensembles with trajectory sampling，PETS）** 是一种使用 MPC 的基于模型的强化学习算法，其关于 **环境模型的构建** 便会同时考虑到这 **两种不确定性**。假设采用 **神经网络** 来构建 **环境状态转移模型$\hat{P}(s,a)$**， **输入** 是状态动作对，如此便可以：

1. *定义**环境模型**的 **输出** 为下一个状态的高斯分布的均值向量和协方差矩阵，来 **捕捉偶然不确定性**。*
> 如此便有 **环境状态转移模型**：
$$\hat{P}(s_t,a_t)=\mathcal{N}(\mu_\theta(s_t,a_t),\Sigma_\theta(s_t,a_t))$$
- $\mu_{\theta}(s_{t},a_{t})$：高斯分布的**均值**，表示模型对下一状态$(s_{t},a_{t})$的预测值，基于当前状态$s_{t}$，动作$a_{t}$和神经网络参数$\theta$进行计算。例如，在自动驾驶系统中，如果当前状态是汽车的位置和速度，动作是加速或转向角度，那么 $\mu_{\theta}(s_{t},a_{t})$ 可能表示汽车在下一时刻的位置和速度的预测。
- **协方差（Covariance）** 描述了两个随机变量之间的 **相关性**，衡量了一个变量变化时另一个变量是否也会随之变化，及其变化的程度和方向。如果是针对一个变量，则就为 **方差**。
- $\Sigma_{\theta}(s_{t},a_{t})$：当前 **状态$S$**，**动作$A$** 变量之间的**协方差**组成的**矩阵**，描述了模型预测的 **"不确定性"**。对角线上的元素代表每个变量的**方差**，非对角线上的元素表示**协方差**。针对单个随机变量的**方差越大说明越不确定**，针对两个随机变量**协方差越大说明相关性越强**，不确定性传播，不确定性增强。所以如果状态转移是比较简单和可预测的，协方差矩阵的值会较小；如果状态转移比较复杂或者在某些情况下较为难以预测（例如，外部干扰、传感器噪声等），协方差矩阵的值会较大。

> 定义神经网络的 **损失函数**：
$$\mathcal{L}(\theta)=\sum_{n=1}^N[\mu_\theta(s_n,a_n)-s_{n+1}]^T\Sigma_\theta^{-1}(s_n,a_n)[\mu_\theta(s_n,a_n)-s_{n+1}]+\log\det\Sigma_\theta(s_n,a_n)$$

- **预测误差项**：$[\mu_\theta(s_n,a_n)-s_{n+1}]^T\Sigma_\theta^{-1}(s_n,a_n)\left[\mu_\theta(s_n,a_n)-s_{n+1}\right]$ 衡量了 **模型预测的状态** $(\mu_\theta(s_n,a_n))$与 **真实状态$(s_{n+1})$** 之间的差异。**协方差矩阵的逆**$\Sigma_\theta^{-1}(s_n,a_n)$通常称为**精度矩阵**，它的作用是与协方差矩阵相对的，作用类似于 **置信度**：如果模型对某些状态转移的预测越确定（协方差矩阵小，协方差矩阵的逆大），对状态变量之间关系的信任度较高，则预测误差对损失函数的贡献较大；如果模型对某些状态转移的预测越不确定（协方差矩阵大，协方差矩阵的逆小），对状态变量之间关系的信任度较低，误差对损失函数的贡献会较小。
- **行列式** 表示的是数据分布的 **体积或扩展程度**。
- **正则化项**：$\log\det\Sigma_\theta(s_n,a_n)$ 通过惩罚大的协方差矩阵的 **行列式**，限制了系统在预测时的  **“扩展性”**，从而帮助模型在多个变量之间**保持平衡**，防止某些不确定性过大的维度影响损失的计算。

>通常 **协方差矩阵**$\Sigma_\theta\in\mathbb{R}^{d\times d}$只考虑对角元素即方差，此时为 **对角矩阵**， **损失函数** 被简化为：
$$\mathcal{L}(\theta)=\sum_{n=1}^N\sum_{i=1}^d[\frac{(\mu_{\theta,i}(s_n,a_n)-s_{n+1,i})^2}{\sigma_{\theta,i}^2}+\log\sigma_{\theta,i}^2]$$
- 减少参数、提高计算效率、保持训练稳定性

2. *用 **集成（ensemble）** 方法：构建$B$个网络框架一样的神经网络，但参数随机初始化采用不同的方式，并且训练时，随机采样不同的真实数据顺序来训练，来 **捕捉认知不确定性**。*

***最终进行模型的预测，评估每条轨迹的价值：***
![PETS算法利用各个环境模型选取动作](Illustrations/PETS算法利用各个环境模型选取动作.png)